# Converting input rootfiles into DataFrame

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import uproot
import awkward as ak

In [2]:
PWD = '${GUNDAM_INPUTS_DIR}/CovMat'

## Configurations (For MakeTree)

In [3]:
TreeMakerFuncName = 'MakeFSICovTree'

MCSampleName = 'NuMI_MC_Nu_Phase2_2023ANuMIReproc_CAFTypeCommonRemerge_ReNuSyst11Aug24'
MCSampleName = 'NuMI_MC_Nu_Phase2_2023ANuMIReproc_CAFTypeCommonRemerge_ReNuSyst11Oct24'

InputBaseDir = f'{PWD}/inputfiles'
InputFileName = f'{TreeMakerFuncName}_{MCSampleName}.root'
InputFilePath = f'{InputBaseDir}/{InputFileName}'

TreeName = 'AllSamples_NoCut/trueEvents'
FriendTreeNames = [
    'AllSamples_NoCut/trueEvents_NSigmas',
    'AllSamples_NoCut/trueEvents_NUniverses',
]

# Output filename
OutputName = 'dfs_%s.pkl'%(TreeMakerFuncName)

##  MC input tree

In [4]:
with uproot.open(InputFilePath) as file:

    # Reco Tree
    Branches = []
    for br in file[TreeName].branches:
        Branches.append(br.name)
    df = file[TreeName].arrays(Branches, library='pd')

    for FriendTreeName in FriendTreeNames:
        FriendBranches = []
        for br in file[FriendTreeName].branches:
            FriendBranches.append(br.name)
        df_Friend = file[FriendTreeName].arrays(FriendBranches, library='pd')

        # Merge df_MC_Friend into DFs[MCSampleAlias]
        df = pd.concat([df, df_Friend], axis=1)
        
        # Delete df_MC_Friend from memory
        del df_Friend



## Export

In [5]:
import pickle
import os

# Export DataFrame and attributes to a single pickle file
with open('%s/%s'%(InputBaseDir, OutputName), 'wb') as f:
    pickle.dump(
        {
            'df': df,
        },
        f
    )